<a href="https://colab.research.google.com/github/nathalyAlarconT/GenAI_Workshops/blob/main/MultiAgents_Gemini_CrewAI_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents that can query and connect to APIs
In this tutorial, we will use Gemini from Google AI Studio and Crew AI to connect an explore some APIs.


In [1]:
%%capture
!pip install --upgrade --quiet  langchain-core langchain_google_genai
!pip install --quiet crewai crewai-tools

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import os

Generate your Gemini API Key in https://aistudio.google.com/apikey

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GoogleAIStudio')

# GOOGLE_API_KEY = ""

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

# Crew AI Setup

In [4]:
import os
import requests
from crewai import Agent, Task, Crew, LLM, Process
from crewai.tools import BaseTool
from typing import Any, Dict

## Gemini Model Configuration

In [7]:
gemini_llm = LLM(
              model='gemini/gemini-2.5-flash',
              api_key=GOOGLE_API_KEY
            )

## APIs Tools configuration

In [5]:
# Custom Tool for Searching Products
class ProductSearchTool(BaseTool):
    name: str = "DummyJSON Product Search"
    description: str = "Searches for products in the DummyJSON API based on a query."

    def _run(self, product_name: str = "phone") -> Dict:
        url = f"https://dummyjson.com/products/search?q={product_name}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return {"products": data.get("products", []), "total": data.get("total", 0)}
        return {"error": "Failed to fetch products"}

# Custom Tool for Fetching Product Details
class ProductDetailTool(BaseTool):
    name: str = "DummyJSON Product Detail"
    description: str = "Fetches detailed information for a specific product by ID from the DummyJSON API."

    def _run(self, product_id: str) -> Dict:
        response = requests.get(f"https://dummyjson.com/products/{product_id}")
        if response.status_code == 200:
            return response.json()
        return {"error": f"Failed to fetch details for product ID {product_id}"}


In [6]:
# Instantiate Tools
search_tool = ProductSearchTool()
detail_tool = ProductDetailTool()

## Agents Definition

In [8]:
searcher = Agent(
    role="Product Searcher",
    goal="Search for products matching a query and select one for detailed analysis.",
    backstory="An expert in e-commerce product discovery, skilled at filtering and selecting relevant products from API data.",
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm
)

detailer = Agent(
    role="Product Detailer",
    goal="Fetch and summarize detailed information for a specific product.",
    backstory="A specialist in extracting and summarizing product details from e-commerce APIs.",
    verbose=True,
    allow_delegation=False,

    llm=gemini_llm
)

## Tasks Definition

In [9]:
search_task = Task(
    description="Search for products in the DummyJSON API using the query : '{product_name}'. Return a list of products with their IDs, titles, and prices, and select the first product for further analysis.",
    expected_output="A JSON object containing a list of products and the ID of the selected product.",
    agent=searcher,
    tools=[search_tool],
)

detail_task = Task(
    description="Fetch detailed information for the selected product ID from the DummyJSON API and summarize the title, price, brand, and description.",
    expected_output="A text summary of the selected product's title, price, brand, and description.",
    agent=detailer,
    tools=[detail_tool],
)

## Crew Setup

In [10]:
crew = Crew(
    agents=[searcher, detailer],
    tasks=[search_task, detail_task],
    process=Process.sequential,
    llm=gemini_llm,
    verbose=True
)

# Crew Execution

In [12]:
inputs = {"product_name": "kiwi"}
result = crew.kickoff(inputs = inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a0cb9e62-8673-4f44-a4b5-6605ed6e3dac                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Searcher                                                                                        │
│                                                                                                                 │
│  Task: Search for products in the DummyJSON API using the query : 'kiwi'. Return a list of products with their  │
│  IDs, titles, and prices, and select the first product for further analysis.                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Searcher                                                                                        │
│                                                                                                                 │
│  Thought: Action: DummyJSON Product Search                                                                      │
│                                                                                                                 │
│  Using Tool: DummyJSON Product Search                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"product_name\": \"kiwi\"}"                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'id': 30, 'title': 'Kiwi', 'description': 'Nutrient-rich kiwi, perfect for snacking or adding a tropical      │
│  twist to your dishes.', 'category': 'groceries', 'price': 2.49, 'discountPercentage': 15.22, 'rating': 4.93,   │
│  'stock': 99, 'tags': ['fruits'], 'sku': 'GRO-BRD-KIW-030', 'weight': 5, 'dimensions': {'width': 19.4,          │
│  'height': 18.67, 'depth': 17.13}, 'warrantyInformation': '6 months warranty', 'shippingInformation': 'Ships    │
│  overnight', 'availabilityStatus': 'In Stock', 'reviews': [{'rating': 4, 'comment': 'Highly recommended!',      │
│  'date': '2025-04-30T09:41:02.053Z', 'reviewerName': 'Emily Brown', 'reviewerEmail':                            │
│  'emily.brown@x.dummyjson.com'}, {'rating': 2, 'comment': 'Would not buy again!', 'date':                       │
│  '2025-04-30T09:41:02.053Z', 'reviewerName': 'Jackson Morales', 'reviewerEmail':                                │
│  'jackson.morales@x.dummyjson.com'}, {'rating': 4, 'comment': 'Fast shipping!', 'date':                         │
│  '2025-04-30T09:41:02.053Z', 'reviewerName': 'Nora Russell', 'reviewerEmail':                                   │
│  'nora.russell@x.dummyjson.com'}], 'returnPolicy': '7 days return policy', 'minimumOrderQuantity': 30, 'meta':  │
│  {'createdAt': '2025-04-30T09:41:02.053Z', 'updatedAt': '2025-04-30T09:41:02.053Z', 'barcode':                  │
│  '2530169917252', 'qrCode': 'https://cdn.dummyjson.com/public/qr-code.png'}, 'images':                          │
│  ['https://cdn.dummyjson.com/product-images/groceries/kiwi/1.webp'], 'thumbnail':                               │
│  'https://cdn.dummyjson.com/product-images/groceries/kiwi/thumbnail.webp'}                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Product Detailer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Title: Kiwi, Price: 2.49, Brand: Not available, Description: Nutrient-rich kiwi, perfect for snacking or       │
│  adding a tropical twist to your dishes.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 114f7f0b-55da-420e-b4fb-b48e519b3a45                                                                     │
│  Agent: Product Detailer                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a0cb9e62-8673-4f44-a4b5-6605ed6e3dac                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Title: Kiwi, Price: 2.49, Brand: Not available, Description: Nutrient-rich kiwi, perfect for     │
│  snacking or adding a tropical twist to your dishes.                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Results

In [13]:
print(result)

Title: Kiwi, Price: 2.49, Brand: Not available, Description: Nutrient-rich kiwi, perfect for snacking or adding a tropical twist to your dishes.


In [14]:
result

CrewOutput(raw='Title: Kiwi, Price: 2.49, Brand: Not available, Description: Nutrient-rich kiwi, perfect for snacking or adding a tropical twist to your dishes.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description="Search for products in the DummyJSON API using the query : 'kiwi'. Return a list of products with their IDs, titles, and prices, and select the first product for further analysis.", name=None, expected_output='A JSON object containing a list of products and the ID of the selected product.', summary='Search for products in the DummyJSON API using the query...', raw='{"products": [{"id": 30, "title": "Kiwi", "price": 2.49}], "selected_product_id": 30}', pydantic=None, json_dict=None, agent='Product Searcher', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='Fetch detailed information for the selected product ID from the DummyJSON API and summarize the title, price, brand, and description.', name=None, expected_output="A text summary of the sel